In [ ]:
import tweepy as tw
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sn
import json
import os
import random

In [ ]:
# Credenciales
consumer_key= ""
consumer_secret= ""
access_token= ""
access_token_secret= ""

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [ ]:
list_accounts = ["EPeconomia", "abceconomia",
                "EFEeconomia", "ExpansionBolsa", 
                 "expansioncom", "elEconomistaes", 
                 "elpais_economia"]
tweets = []

for account in list_accounts:
    tweets.append(api.user_timeline(screen_name=account, 
                               # 300 is the maximum allowed count
                               count=300,
                               include_rts = False,
                               # Necessary to keep full_text 
                               # otherwise only the first 140 words are extracted
                               tweet_mode = 'extended'
                               ))
    
if (not os.path.exists('json_files')):
    os.mkdir('json_files')

In [ ]:
# Creamos los ficheros json

for elem in range(0,len(tweets)):
    list_tweets=[]
    for js in range(0,len(tweets[elem])):
        date = tweets[elem][js]._json['created_at']
        text = tweets[elem][js]._json['full_text']
        tweet = {
            'Date': date,
            'Tweet': text,
            'Target': '',
            'Sent_Target': '',
            'Sent_Sociedad': '',
            'Sent_Empresas': ''
        }
        list_tweets.append(tweet)
        
    account = "@" + tweets[elem][0]._json['user']['screen_name']

    usuario = {
        'Account': account,
        'Tweets': list_tweets
    }
        
    filename = tweets[elem][0]._json['user']['name'].replace(" ","_") + ".json"
    
    with open ('json_files/' + filename, 'w') as f:
        json.dump (usuario, f, indent = 4)

In [ ]:
path_to_json = 'json_files/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

primerUsuario = True

for js in json_files:
        with open(os.path.join(path_to_json, js), "r") as json_file:
            usuario = json.load(json_file)
            if primerUsuario:
                df = pd.json_normalize(usuario, meta= ["Account"], record_path =["Tweets"])
                primerUsuario = False
            else:
                df_account = pd.json_normalize(usuario, meta= ["Account"], record_path =["Tweets"])
                df = df.append(df_account, ignore_index = True)

In [ ]:
path_to_json = 'json_files/expansioncom.json'
#json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

primerUsuario = True

with open(path_to_json, "r") as json_file:
            usuario = json.load(json_file)
            if primerUsuario:
                df = pd.json_normalize(usuario, meta= ["Account"], record_path =["Tweets"])
                primerUsuario = False
            else:
                df_account = pd.json_normalize(usuario, meta= ["Account"], record_path =["Tweets"])
                df = df.append(df_account, ignore_index = True)


In [ ]:
df.to_csv("data_twitter/tweets_finanzas.csv")